In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import mxnet as mx
from mxnet import nd
from mxnet import image
from resnet import ResNet164_v2
from densenet import DenseNet
from mxnet.gluon.data.vision import ImageFolderDataset
from mxnet.gluon.data import DataLoader

### data transform and augmentation

In [3]:
def transform_train(data, label):
    im = data.asnumpy()
    im = np.pad(im, ((4, 4), (4, 4), (0, 0)), mode='constant', constant_values=0)
    im = nd.array(im, dtype='float32') / 255
    auglist = image.CreateAugmenter(data_shape=(3, 32, 32), resize=0, rand_mirror=True,
                                    rand_crop=True,
                                   mean=np.array([0.4914, 0.4822, 0.4465]),
                                   std=np.array([0.2023, 0.1994, 0.2010]))
    for aug in auglist:
        im = aug(im)
    im = nd.transpose(im, (2, 0, 1)) # channel x width x height
    return im, nd.array([label]).astype('float32')

def transform_test(data, label):
    im = data.astype('float32') / 255
    auglist = image.CreateAugmenter(data_shape=(3, 32, 32), mean=np.array([0.4914, 0.4822, 0.4465]),
                                   std=np.array([0.2023, 0.1994, 0.2010]))
    for aug in auglist:
        im = aug(im)
    im = nd.transpose(im, (2, 0, 1))
    return im, nd.array([label]).astype('float32')

In [4]:
train_ds = ImageFolderDataset('./data/train_data/', transform=transform_train)
valid_ds = ImageFolderDataset('./data/valid_data/', transform=transform_test)
train_valid_ds = ImageFolderDataset('./data/train_valid/', transform=transform_train)
test_ds = ImageFolderDataset('./data/TestSet/', transform=transform_test)

In [5]:
test_data = DataLoader(test_ds, batch_size=128, shuffle=False, last_batch='keep')

### load trained resnet164 model and densnet model

In [6]:
ctx = mx.gpu(1)

In [7]:
net1 = ResNet164_v2(10)
net1.load_params('./resnet.params', ctx=ctx)
net1.hybridize()

In [8]:
net2 = DenseNet(growthRate=12, depth=100, reduction=0.5, bottleneck=True, nClasses=10)
net2.load_params('./densenet.params', ctx=ctx)
net2.hybridize()

In [9]:
import pandas as pd

preds = []
for data, _ in test_data:
    data = data.as_in_context(ctx)
    output1 = nd.softmax(net1(data))
    output2 = nd.softmax(net2(data))
    output = 0.9523 * output1 + 0.9539 * output2
    pred_label = output.argmax(1)
    preds.extend(pred_label.astype(int).asnumpy())

sorted_ids = list(range(1, len(test_ds) + 1))
sorted_ids.sort(key = lambda x:str(x))

df = pd.DataFrame({'id': sorted_ids, 'label': preds})
df['label'] = df['label'].apply(lambda x: train_ds.synsets[x])
df.to_csv('ensemblesubmission.csv', index=False)